<a href="https://colab.research.google.com/github/nemtallahdaher/Colab/blob/main/cis524final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/python3

from datetime import timedelta
from datetime import datetime
import re
import sys


def printerror(errstr,line,col):
    print("Line "+str(line)+", column "+str(col)+", ",errstr)
    print("Compilation failed.")
    exit()

def printdebug(errstr):
    if debugme == 'y':
        print(errstr)

def syntaxerror(line):
    s2vardef = "^var +"
    s2varass = revarname+" *\= *"
    s2for = "^for *\("
    if re.match(s2vardef,line):
        return(vardef(line))
    elif re.match(s2for,line):
        return(fordef(line))
    else:
        return(varass(line))

def cleanwords(words):
    newwords = []
    for word in words:
        if word:
            newwords.append(word.strip())
    printdebug(newwords)
    return(newwords)

def vardef(line):
    printdebug("in vardef")
    vardefstate = [revarname,'\;']
    words = cleanwords(re.split('([ +\=\;])',line))
    return(finderror(vardefstate,1,words,line))

def varass(line):
    printdebug("in varass")
    varassstate = ["("+revarname+"|"+reconst+")",'\+|\;']
    words = cleanwords(re.split(r' +|([\+\=\;])',line))
    return(finderror(varassstate,2,words,line))

def fordef(line):
    printdebug("in fordef")
    forstate = ["("+revarname+"|"+reconst+")",'\=|\<|\>|\;']
    forstate2 = [revarname,'\+\+']
    forstate3 = ['\)','\{']
    words = cleanwords(re.split(r' +|([\=\+\<\>\;\(\)\{])',line))
    column = finderror(forstate,3,words,line,8)
    if column:
        return(column)
    column = finderror(forstate2,11,words,line,2)
    if column:
        return(column)
    column = finderror(forstate3,13,words,line,2)
    if column:
        return(column)




def finderror(repat,restart,words,line,recurse=9999):
    i = 0
    for word in words[restart:]:
        if i < recurse:
            x = re.match(repat[i % len(repat)],word)
            if x:
                i += 1
            else:
                return(line.index(word)+1)


def lparse(fname):                        #define function
    beginproc = 0
    linenum = 0                        #variable to count line numbers
    stack = []
    var_dict = dict()
    with open(fname, 'r') as f:        #read file
        for oneline in f:              #look at one line at a time
            line = oneline.rstrip().lstrip()

            if(line == ""):
                continue

            linenum += 1           #increment line variable
            if re.fullmatch(revardef, line):
                x = re.fullmatch(revardef, line)
                if x[1] in var_dict:
                    printerror("error: var already defined",linenum,0)
                else:
                    var_dict[x[1]] = 1
                    printdebug("variable definition")

            elif re.fullmatch(refor, line):
                printdebug("for statement")
                beginproc +=1
                x = re.search(refordef,line)
                if x[1] in var_dict:
                    printerror("error: variable already defined",linenum,0)
                else:
                    var_dict[x[1]] = 1
                    stack.append(x[1])

            elif re.fullmatch(revarass, line):
                printdebug("variable assignment")
                x = re.fullmatch(revarass, line)
                z = x.groups()
                for i in z:
                    if i:
                        if re.fullmatch(revarname, i):
                            if i in var_dict:
                                printdebug("variable found "+i)
                            else:
                                printerror("error: var not previously defined",linenum,0)

            elif re.fullmatch(recurlend,line):
                beginproc -= 1
                printdebug("curly end")
                if beginproc < 0:
                    printerror("error: unmatched braces",linenum,0)
                del var_dict[stack.pop()]

            else:
                column = syntaxerror(line)
                printerror("incorrect grammar",linenum,column)


if __name__ == '__main__':
    revarname = "([a-z][0-1a-z]+)"
    reconst = "[0-1]+"
    revardef = "^var +"+revarname+" *;"
    reexp = "("+revarname+"|"+reconst+")( *\+ *("+revarname+"|"+reconst+"))*"
    revarass = revarname+" *\= *"+reexp+" *;"
    revarcond = revarname+" *[\<\>] *"+reexp+" *;"
    revarinc = revarname+" *((\+\+)|(\-\-))"
    refor = "^for *\( *var +"+revarass+" *"+revarcond+" *"+revarinc+" *\) *\{"
    refordef = "var +"+revarname
    recurlend = " *} *"

    debugme = 'n'
    if len(sys.argv) > 2:
        if sys.argv[2] == 'debug':
            debugme = 'y'

    lparse(sys.argv[1])
    print("Compilation succeeded.")

FileNotFoundError: ignored